In [ ]:
import os
import cv2 as cv
import matplotlib.pyplot as plt

class CascadeObjectDetector():
    '''
    Class for Cascade Object Detection
    '''
    def __init__(self,object_cascade_path):
        '''
        Args:
        object_cascade_path: path for the *.xml defining the parameters 
                for {face, eye, smile, profile} detection algorithm
                source of the haarcascade resource is: 
                https://github.com/opencv/opencv/tree/master/data/haarcascades
        Returns:
            None
        '''

        self.object_cascade=cv.CascadeClassifier(object_cascade_path)


    def detect(self, image, scale_factor=1.3,
               min_neighbors=5,
               min_size=(20,20)):
        '''
        Function return rectangle coordinates of object for given image
        Args:
            image: image to process
            scale_factor: scale factor used for object detection
            min_neighbors: minimum number of parameters considered during object detection
            min_size: minimum size of bounding box for object detected
        Returns:
            rectangle with detected object
            
        '''
        rects=self.object_cascade.detectMultiScale(image,
                                                scaleFactor=scale_factor,
                                                minNeighbors=min_neighbors,
                                                minSize=min_size)
        return rects

class FaceObjectDetector():
    '''
    Class for Face Object Detection
    '''
    def __init__(self, face_detection_folder):
        '''
        Args:
        face_detection_folder: path for folder where the *.xmls
                for {face, eye, smile, profile} detection algorithm
        Returns:
            None
        '''

        self.path_cascade=face_detection_folder
        self.frontal_cascade_path= os.path.join(self.path_cascade,'haarcascade_frontalface_default.xml')
        self.eye_cascade_path= os.path.join(self.path_cascade,'haarcascade_eye.xml')
        self.profile_cascade_path= os.path.join(self.path_cascade,'haarcascade_profileface.xml')
        self.smile_cascade_path= os.path.join(self.path_cascade,'haarcascade_smile.xml')

        #Detector object created
        # frontal face
        self.face_detector=CascadeObjectDetector(self.frontal_cascade_path)
        # eye
        self.eyes_detector=CascadeObjectDetector(self.eye_cascade_path)
        # profile face
        self.profile_detector=CascadeObjectDetector(self.profile_cascade_path)
        # smile
        self.smile_detector=CascadeObjectDetector(self.smile_cascade_path)

    def detect_objects(self,
                       image, 
                       scale_factor,
                       min_neighbors, 
                       min_size, 
                       show_smile=False):
        '''
        Objects detection function
        Identify frontal face, eyes, smile and profile face and display the detected objects over the image
        Args:
            image: the image extracted from the video
            scale_factor: scale factor parameter for `detect` function of CascadeObjectDetector object
            min_neighbors: min neighbors parameter for `detect` function of CascadeObjectDetector object
            min_size: minimum size parameter for f`detect` function of CascadeObjectDetector object
            show_smile: flag to activate/deactivate smile detection; set to False due to many false positives
        Returns:
            None
        '''

        image_gray=cv.cvtColor(image, cv.COLOR_BGR2GRAY)


        eyes=self.eyes_detector.detect(image_gray,
                       scale_factor=scale_factor,
                       min_neighbors=min_neighbors,
                       min_size=(int(min_size[0]/2), int(min_size[1]/2)))

        for x, y, w, h in eyes:
            #detected eyes shown in color image
            cv.circle(image,(int(x+w/2),int(y+h/2)),(int((w + h)/4)),(0, 0,255),3)

        # deactivated by defauld due to many false positive
        if show_smile:
            smiles=self.smile_detector.detect(image_gray,
                          scale_factor=scale_factor,
                          min_neighbors=min_neighbors,
                          min_size=(int(min_size[0]/2), int(min_size[1]/2)))

            for x, y, w, h in smiles:
               #detected smiles shown in color image
               cv.rectangle(image,(x,y),(x+w, y+h),(0, 0,255),3)


        profiles=self.profile_detector.detect(image_gray,
                       scale_factor=scale_factor,
                       min_neighbors=min_neighbors,
                       min_size=min_size)

        for x, y, w, h in profiles:
            #detected profiles shown in color image
            cv.rectangle(image,(x,y),(x+w, y+h),(255, 0,0),3)

        faces=self.face_detector.detect(image_gray,
                       scale_factor=scale_factor,
                       min_neighbors=min_neighbors,
                       min_size=min_size)

        for x, y, w, h in faces:
            #detected faces shown in color image
            cv.rectangle(image,(x,y),(x+w, y+h),(0, 255,0),3)

        # image
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(111)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        ax.imshow(image)  
    
    
    def extract_image_objects(self,
                              video_file,
                              data_folder,
                              video_set_folder,
                              show_smile=False
                             ):
        '''
        Extract one image from the video and then perform face/eyes/smile/profile detection on the image
        Args:
            video_file: the video from which to extract the image from which we extract the face
            data_folder: folder with the data
            video_set_folder: folder with the video set
            show_smile: show smile (False by default)
        Returns:
            None
        '''
        video_path = os.path.join(data_folder, video_set_folder,video_file)
        capture_image = cv.VideoCapture(video_path) 
        ret, frame = capture_image.read()
        #frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        self.detect_objects(image=frame, 
                scale_factor=1.3, 
                min_neighbors=5, 
                min_size=(50, 50),
                show_smile=show_smile)      